In [ ]:
import h5py 
import dxchange
import os
import numpy as np
from holotomocupy.utils import *

## Parameters

In [ ]:
n = 2048
ntheta = 4500
detector_pixelsize = 1.4760147601476e-6 * 2
energy = 17.1
wavelength = 1.24e-09 / energy
focustodetectordistance = 1.217

sx0 = -3.135e-3
z1 = np.array([5.110, 5.464, 6.879, 9.817, 10.372, 11.146, 12.594, 17.209]) * 1e-3 - sx0
z1_ids = np.array([0, 1, 2, 3]) ### note using index starting from 0
str_z1_ids = ''.join(map(str, z1_ids + 1)) 
z1 = z1[z1_ids]
ndist = len(z1)
z2 = focustodetectordistance - z1

distances = (z1 * z2) / focustodetectordistance
magnifications = focustodetectordistance / z1
norm_magnifications = magnifications / magnifications[0]
voxelsizes = np.abs(detector_pixelsize / magnifications)
voxelsize = voxelsizes[0]

path = '/data2/vnikitin/brain/20251115/'
pfile = 'Y350a_HT_20nm_8dist'
path_out = '/data2/vnikitin/brain_rec/20251115/Y350a'
file_out = f'data{str_z1_ids}.h5'


### Parse files and save everything to h5

In [ ]:
def find_angle(filename):    
    with open(filename, 'r', encoding="latin-1") as file:
        for line in file:
            if "motor_pos" in line:
                return float(line.split()[3])

os.makedirs(path_out, exist_ok=True)
with h5py.File(f'{path_out}/{file_out}', 'w') as fid:
    data = [fid.create_dataset(f'/exchange/data{k}', shape=(ntheta, n, n), dtype='uint16') for k in range(ndist)]
    data_white0 = [fid.create_dataset(f'/exchange/data_white_start{k}', shape=(20, n, n), dtype='uint16') for k in range(ndist)]
    data_white1 = [fid.create_dataset(f'/exchange/data_white_end{k}', shape=(20, n, n), dtype='uint16') for k in range(ndist)]
    data_dark = [fid.create_dataset(f'/exchange/data_dark{k}', shape=(20, n, n), dtype='uint16') for k in range(ndist)]
    
    theta = fid.create_dataset('/exchange/theta', shape=(ntheta, ndist), dtype='float32')
    shifts = fid.create_dataset('/exchange/shifts', shape=(ntheta, ndist, 2), dtype='float32')
    attrs = fid.create_dataset('/exchange/attrs', shape=(ntheta, ndist, 3), dtype='float32')
    
    fid.create_dataset('/exchange/voxelsize', data=voxelsizes, dtype='float32')
    fid.create_dataset('/exchange/z1', data=z1, dtype='float32')
    fid.create_dataset('/exchange/detector_pixelsize', data=[detector_pixelsize], dtype='float32')
    fid.create_dataset('/exchange/energy', data=[energy], dtype='float32')
    fid.create_dataset('/exchange/focusdetectordistance', data=[focustodetectordistance], dtype='float32')    
    for k in range(ndist):
        dname = f'{path}/{pfile}_{z1_ids[k] + 1}_'
        shifts[:, k] = np.loadtxt(f'{dname}/correct.txt')[:ntheta]
        attrs[:, k] = np.loadtxt(f'{dname}/attributes.txt')[:ntheta, :3]

        [n0,n1] = dxchange.read_edf(f'{dname}/ref{0:04}_0000.edf')[0].shape
        sty,endy = n0//2 - n//2,n0//2 + n//2
        stx,endx = n1//2 - n//2,n1//2 + n//2
        for id in range(20):
            data_white0[k][id] = dxchange.read_edf(f'{dname}/ref{id:04}_0000.edf')[0][sty:endy,stx:endx]
            data_white1[k][id] = dxchange.read_edf(f'{dname}/ref{id:04}_{ntheta:04}.edf')[0][sty:endy,stx:endx]
            data_dark[k][id] = dxchange.read_edf(f'{dname}/darkend{id:04}.edf')[0][sty:endy,stx:endx]

        for id in range(ntheta):
            fname = f'{dname}/{pfile}_{z1_ids[k] + 1}_{id:04}.edf'
            ang = find_angle(fname)                    
            
            data[k][id] = dxchange.read_edf(fname)[0][sty:endy,stx:endx]
            theta[id, k] = ang
            if id%100==0:
                print(id,k,theta[id,k],shifts[id, k])